<a href="https://colab.research.google.com/github/Tomawock/NLP_Attack/blob/main/Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Import vari e inizializzazione pacchetti**

In [ ]:
pip install textattack

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# **AMAZON**

In [15]:
dataset = pd.read_csv('Reviews.csv')
dataset.columns

Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')

In [16]:
from textattack.augmentation import WordNetAugmenter
augmenter = WordNetAugmenter(pct_words_to_swap=0.2, transformations_per_example=5)
s = dataset.Text[1]
augmenter.augment(s)

['Product arrived judge as Jumbo salt-cured Peanuts...the peanuts were actually modest sized unsalted. Not surely if this was an error or if the vendor designate to comprise the product as "Jumbo".',
 'Product come labeled as gargantuan Salted Peanuts...the peanuts were actually belittled sized unsalted. Not sure if this was an misplay or if the vendor intended to comprise the product as "jumbo".',
 'merchandise arrived mark as Jumbo Salted groundnut...the peanuts were actually belittled sized unsalted. Not sure if this was an error or if the trafficker think to represent the product as "Jumbo".',
 'ware arrived labeled as giant Salted Peanuts...the peanuts were actually belittled sized unsalted. Not sure if this was an fault or if the vendor mean to represent the product as "giant".',
 'ware arrived tag as Jumbo salt Peanuts...the peanuts were actually pocket-size sized unsalted. Not sure if this was an error or if the vendor intended to correspond the product as "gargantuan".']

# **ATE ABISTA**

> WORD NET AUGMENTER da fare da circa zero unsado la classe WordSwapWordNet(lang=italian or it)



In [11]:
# import transformations, contraints, and the Augmenter
from textattack.transformations import WordSwapWordNet

from textattack.constraints.pre_transformation import RepeatModification
from textattack.constraints.pre_transformation import StopwordModification

from textattack.augmentation import Augmenter

In [18]:
# Set up transformation using CompositeTransformation()
transformation = WordSwapWordNet(language='ita')
# Set up constraints
constraints = [RepeatModification(), StopwordModification()]
# Create augmenter with specified parameters
augmenter = Augmenter(transformation=transformation, constraints=constraints, pct_words_to_swap=0.7, transformations_per_example=5)
s = 'Vorrei mangiare la carbonara, ma il ristorante è al completo. Penso mangierò una pizza dal mio kebabbaro preferito!'
# Augment!
augmenter.augment(s)

['Vorrei cibo la carbonara, ma il trattoria è al grandissimo. Penso mangierò una bobina dal mio kebabbaro beniamino!',
 'Vorrei cibo la carbonara, ma il trattoria è al insieme. Penso mangierò una mattone dal mio kebabbaro coccolo!',
 'Vorrei cibo la carbonara, ma il trattoria è al insieme. Penso mangierò una noioso dal mio kebabbaro favorito!',
 'Vorrei pasto la carbonara, ma il restaurant è al totale. Penso mangierò una lagna dal mio kebabbaro beniamino!',
 'Vorrei pasto la carbonara, ma il trattoria è al incondizionato. Penso mangierò una noia dal mio kebabbaro migliore!']